# Download tiles automatically from swissimage

In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import geopandas as gpd
from tqdm import tqdm

In [ ]:
# test
url_img = "https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2023_2569-1152/swissimage-dop10_2023_2569-1152_0.1_2056.tif"

year = 2025
url_img = f"https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_{year}_2569-1152/swissimage-dop10_{year}_2569-1152_0.1_2056.tif"
while requests.get(url_img).status_code != 200:
    year -= 1
    url_img = f"https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_{year}_2569-1152/swissimage-dop10_{year}_2569-1152_0.1_2056.tif"

img_data = requests.get(url_img).content
with open('../data/test_downloading/test.tif', 'wb') as handler:
    handler.write(img_data)

200
https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2025_2569-1152/swissimage-dop10_2025_2569-1152_0.1_2056.tif
<Response [404]>
https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2024_2569-1152/swissimage-dop10_2024_2569-1152_0.1_2056.tif
<Response [404]>


### Downloading from shapefiles

In [11]:
cantons = gpd.read_file('../data/test_downloading/swissboundaries/swissBOUNDARIES3D_1_5_TLM_KANTONSGEBIET.shp')
# print(cantons.head())
Vaud = cantons[cantons.NAME == 'Vaud']
# print(Vaud)
print(Vaud.bounds)
print(Vaud.bounds.minx.values[0])
Emin = int(Vaud.bounds.minx.values[0] // 1000)
Emax = int((Vaud.bounds.maxx.values[0] + 1) // 1000)
Nmin = int(Vaud.bounds.miny.values[0] // 1000)
Nmax = int((Vaud.bounds.maxy.values[0] + 1) // 1000)
print(Emin)
print(Emax)
print(Nmin)
print(Nmax)

           minx         miny         maxx         maxy
15  2494306.344  1115148.823  2585462.064  1204075.563
2494306.344
2494
2585
1115
1204


#### Load tiles locations

In [12]:
tiles_locs = gpd.read_file(r"..\data\test_downloading\shapefile_tiles_locs\ch.swisstopo.images-swissimage-dop10.metadata.shp")
ids = tiles_locs.id.values
E = [x.split('_')[0] for x in ids]
N = [x.split('_')[1] for x in ids]
EN = [[int(x), int(y)] for x,y in zip(E,N)]
VD = [x for x in EN if Emin <= x[0] <= Emax and Nmin <= x[1] <= Nmax]
print("Number of tiles in Switzerland: ", len(EN))
print("Number of tiles in Vaud: ", len(VD))

Number of tiles in Switzerland:  42691
Number of tiles in Vaud:  5945


In [38]:
cantons = gpd.read_file('../data/test_downloading/swissboundaries/swissBOUNDARIES3D_1_5_TLM_KANTONSGEBIET.shp')
cantons_dict = {}
print("Number of tiles in Switzerland: ", len(EN))
estimated_time = round(len(EN)/20/60, 2)
estimated_size = round(len(EN) * 0.054, 2)
print(f"  - estimated time to download: {estimated_time}hrs")
print(f"  - estimated size: {estimated_size}Go\n")

for canton in cantons.itertuples():
    name = canton.NAME
    cantons_dict[name] = {}

    Emin = int(canton.geometry.bounds[0] // 1000)
    Emax = int((canton.geometry.bounds[2] + 1) // 1000)
    Nmin = int(canton.geometry.bounds[1] // 1000)
    Nmax = int((canton.geometry.bounds[3] + 1) // 1000)
    
    lst_tiles = [x for x in EN if Emin <= x[0] <= Emax and Nmin <= x[1] <= Nmax]

    cantons_dict[name]['tiles'] = lst_tiles
    estimated_time = round(len(lst_tiles)/20/60, 2)
    estimated_size = round(len(lst_tiles) * 0.054, 2)
    print(f"{name} - Number of tiles: {len(lst_tiles)} \n  - estimated time to download: {estimated_time}hrs \n  - estimated size: {estimated_size}Go")


Number of tiles in Switzerland:  42691
  - estimated time to download: 35.58hrs
  - estimated size: 2305.31Go

Genève - Number of tiles: 400 
  - estimated time to download: 0.33hrs 
  - estimated size: 21.6Go
Thurgau - Number of tiles: 2085 
  - estimated time to download: 1.74hrs 
  - estimated size: 112.59Go
Valais - Number of tiles: 9021 
  - estimated time to download: 7.52hrs 
  - estimated size: 487.13Go
Aargau - Number of tiles: 2816 
  - estimated time to download: 2.35hrs 
  - estimated size: 152.06Go
Schwyz - Number of tiles: 1833 
  - estimated time to download: 1.53hrs 
  - estimated size: 98.98Go
Zürich - Number of tiles: 2812 
  - estimated time to download: 2.34hrs 
  - estimated size: 151.85Go
Obwalden - Number of tiles: 936 
  - estimated time to download: 0.78hrs 
  - estimated size: 50.54Go
Fribourg - Number of tiles: 3200 
  - estimated time to download: 2.67hrs 
  - estimated size: 172.8Go
Glarus - Number of tiles: 1290 
  - estimated time to download: 1.07hrs 
  

#### Download tiles for Genève

In [21]:
def download_tile(E, N, dest, suffixe=''):
    year = 2025
    url_img = f"https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_{year}_{E}-{N}/swissimage-dop10_{year}_{E}-{N}_0.1_2056.tif"
    count_down = 0
    while requests.get(url_img).status_code != 200:
        if count_down > 10:
            print(f"Could not find a tile for coordinates {E}-{N}")
            return
        year -= 1
        count_down += 1
        url_img = f"https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_{year}_{E}-{N}/swissimage-dop10_{year}_{E}-{N}_0.1_2056.tif"

    img_data = requests.get(url_img).content
    file_src = os.path.join(dest, f"tile_{E}-{N}_{year}_{suffixe}.tif")
    with open(file_src, 'wb') as handler:
        handler.write(img_data)

In [23]:
for _, tile_loc in tqdm(enumerate(cantons_dict['Genève']['tiles']), total=len(cantons_dict['Genève']['tiles'])):
    download_tile(tile_loc[0], tile_loc[1], '../data/test_downloading')

100%|██████████| 400/400 [19:15<00:00,  2.89s/it]
